# Definisanje ulaznih podataka (Inputi)

## 1. Uvoz biblioteka i definisanje putanja

U ovoj sekciji se uvozi arcpy biblioteka i definišu se sve ulazne i izlazne putanje kako bi kod bio pregledan i lako promenljiv.

In [60]:
import arcpy
import os

In [61]:
# Putanja do geobaze projekta
putanja_do_baze = r"C:\\Users\\LENOVO\\Desktop\\FAKS\\IV GODINA\\NMPOP\\Sombor_objekti"

In [62]:
# Putanja do shapefile-a opštine Sombor 
opstina_sombor_shp = r"C:\Users\LENOVO\Desktop\FAKS\IV GODINA\NMPOP\Sombor_objekti\Opština\Op_tina.shp"

In [63]:
# Putanja do shapefile-a naselja Srbije 
naselja_srbije_shp = r"C:\\Users\\LENOVO\\Desktop\\FAKS\\IV GODINA\\NMPOP\\Sombor_objekti\\Naselja\\Naseljeno mesto.shp"

In [64]:
# Folder gde su ekstraktovani OSM podaci sa Geofabrika
osm_folder = r"C:\\Users\\LENOVO\\Desktop\\FAKS\\IV GODINA\\NMPOP\\Sombor_objekti\\gisSrbija"

In [65]:
# Matični broj opštine Sombor 
maticni_broj_opstine_sombor = "80381" 

In [66]:
# Podešavanje radnog okruženja
arcpy.env.workspace = putanja_do_baze
arcpy.env.overwriteOutput = True

In [67]:
print("Putanja do baze:", putanja_do_baze)
print("Spremno za rad.")

Putanja do baze: C:\\Users\\LENOVO\\Desktop\\FAKS\\IV GODINA\\NMPOP\\Sombor_objekti
Spremno za rad.


# Izdvajanje istraživanog područja (Opština i naselja)

## 2. Izdvajanje istraživanog područja

Koristimo alat Select da izdvojimo poligon opštine Sombor i Select Layer By Location da pronađemo sva naselja na njenoj teritoriji.

In [68]:
# Izdvajanje opštine Sombor
where_clause_opstina = f"grad_matic = {maticni_broj_opstine_sombor}" 
arcpy.analysis.Select(opstina_sombor_shp, "Opstina_Sombor", where_clause_opstina)
print("Feature class 'Opstina_Sombor' je kreiran.")

Feature class 'Opstina_Sombor' je kreiran.


In [69]:
# Izdvajanje naselja koja pripadaju opštini Sombor
# Prvo učitavamo lejer opštine i naselja
arcpy.management.MakeFeatureLayer(naselja_srbije_shp, "Naselja_Srbije_Layer")
arcpy.management.MakeFeatureLayer("Opstina_Sombor", "Opstina_Sombor_Layer")

<Result 'Opstina_Sombor_Layer'>

In [70]:
# Selekcija naselja koja se preklapaju sa opštinom
arcpy.management.SelectLayerByLocation("Naselja_Srbije_Layer", "HAVE_THEIR_CENTER_IN", "Opstina_Sombor_Layer")

<Result 'Naselja_Srbije_Layer'>

In [71]:
# Čuvanje selektovanih naselja u novi feature class-u
arcpy.management.CopyFeatures("Naselja_Srbije_Layer", "Naselja_Sombora")
print("Feature class 'Naselja_Sombora' sa svim naseljima opštine je kreiran.")

Feature class 'Naselja_Sombora' sa svim naseljima opštine je kreiran.


# Isecanje (Clip) OSM podataka i prevođenje

## 3. Isecanje i obrada OSM podataka

Svi relevantni OSM podaci (objekti, putevi) se isecaju prema granicama opštine Sombor. Nakon toga, dodajemo novu kolonu sa nazivima tipova objekata na srpskom jeziku.

In [72]:
# Primer za tačkaste objekte (Points of Interest)
# Putanja do OSM tačkastih objekata
osm_poi_shp = os.path.join(osm_folder, r"C:\Users\LENOVO\Desktop\FAKS\IV GODINA\NMPOP\Sombor_objekti\gisSrbija\gis_osm_pois_free_1.shp") 

# Isecanje (Clip)
arcpy.analysis.Clip(osm_poi_shp, "Opstina_Sombor", "Objekti_Sombor_OSM")
print("OSM objekti isečeni na teritoriju Sombora.")

OSM objekti isečeni na teritoriju Sombora.


In [73]:
# Filtriranje samo relevantnih objekata i prevod 
# Definišemo koje klase nas zanimaju
interesantni_objekti_fclass = [
    'hospital', 'clinic', 'doctors', 'pharmacy', # Zdravstvo
    'school', 'kindergarten', 'university', 'college', # Obrazovanje
    'post_office', # Pošta
    'bank', # Banka
    'police', 'courthouse', 'townhall' # Javne institucije
]

# Pravimo SQL upit za selekciju
where_clause_objekti = "fclass IN ('" + "', '".join(interesantni_objekti_fclass) + "')"


# Selektujemo samo te objekte
arcpy.analysis.Select("Objekti_Sombor_OSM", "Javni_Objekti_Sombor", where_clause_objekti)
print("Izdvojeni su samo relevantni javni objekti.")

Izdvojeni su samo relevantni javni objekti.


In [74]:
# Dodavanje kolone i prevod na srpski
# Dodajemo novo polje "Tip_Objekta"
arcpy.management.AddField("Javni_Objekti_Sombor", "Tip_Obj", "TEXT", field_length=50)

# Definišemo funkciju za prevod
# Ovo je Python kod koji se izvršava unutar Calculate Field alata
kod_za_prevod = """
def prevedi_tip(fclass):
    mapa_prevoda = {
        'hospital': 'Bolnica', 'clinic': 'Klinika', 'doctors': 'Ambulanta', 'pharmacy': 'Apoteka',
        'school': 'Škola', 'kindergarten': 'Vrtić', 'university': 'Fakultet', 'college': 'Viša škola',
        'post_office': 'Pošta',
        'bank': 'Banka',
        'police': 'Policija', 'courthouse': 'Sud', 'townhall': 'Opština/Gradska kuća'
    }
    return mapa_prevoda.get(fclass, 'Ostalo')
"""

# Pokrećemo Calculate Field
arcpy.management.CalculateField("Javni_Objekti_Sombor", "Tip_Obj", "prevedi_tip(!fclass!)", "PYTHON3", kod_za_prevod)
print("Kolona 'Tip_Obj' je popunjena srpskim nazivima.")

# Dodavanje i računanje geometrije (X i Y koordinate)
arcpy.management.AddXY("Javni_Objekti_Sombor")
print("Dodate su X i Y koordinate za svaki objekat.")

Kolona 'Tip_Obj' je popunjena srpskim nazivima.
Dodate su X i Y koordinate za svaki objekat.


# Analiza - Pronalaženje najbližeg objekta

## 4. Analiza: Udaljenost naselja do najbližeg objekta

Za svaki tip objekta (npr. pošta), prvo identifikujemo naselja koja ga nemaju. Zatim, za ta naselja, koristimo alat Near da izračunamo udaljenost do najbliže pošte.

In [75]:
import arcpy
import re

# KORAK 1: Automatsko pronalaženje svih jedinstvenih tipova objekata
# Koristimo kursor da pročitamo sve vrednosti iz kolone 'Tip_Obj'

svi_tipovi_objekata = set()
with arcpy.da.SearchCursor("Javni_Objekti_Sombor", ["Tip_Obj"]) as cursor:
    for row in cursor:
        svi_tipovi_objekata.add(row[0])

lista_tipova = list(svi_tipovi_objekata)
print(f"Pronađeni tipovi objekata za analizu: {lista_tipova}")


# KORAK 2: Petlja koja radi analizu za svaki tip objekta
arcpy.management.MakeFeatureLayer("Naselja_Sombora", "Naselja_Sombora_Layer")
print("\n--- Početak analize dostupnosti za sve tipove objekata ---")

naziv_kolone_u_naseljima = "naselje__1" 

for tip_objekta in lista_tipova:
    print(f"\n>>> Započinjem obradu za tip: '{tip_objekta}'")

    cisti_naziv_tipa = re.sub(r'[^a-zA-Z0-9]', '_', tip_objekta)
    objekti_trenutnog_tipa = f"Objekti_{cisti_naziv_tipa}"
    naselja_sa_objektom_join = f"Naselja_Sa_{cisti_naziv_tipa}_Join"
    naselja_bez_objekta = f"Naselja_Bez_{cisti_naziv_tipa}"

    # 1. Izdvajanje objekata trenutnog tipa
    sql_upit = f"Tip_Obj = '{tip_objekta}'"
    arcpy.analysis.Select("Javni_Objekti_Sombor", objekti_trenutnog_tipa, sql_upit)
    print(f"  - Izdvojeni su objekti tipa '{tip_objekta}'.")

    # 2. Pronalaženje naselja BEZ objekta
    arcpy.analysis.SpatialJoin("Naselja_Sombora_Layer", objekti_trenutnog_tipa, naselja_sa_objektom_join, "JOIN_ONE_TO_ONE", "KEEP_ALL", match_option="INTERSECT")
    arcpy.analysis.Select(naselja_sa_objektom_join, naselja_bez_objekta, "Join_Count = 0")
    print(f"  - Pronađena su naselja bez objekta tipa '{tip_objekta}'.")

    # 3. Izračunavanje udaljenosti
    arcpy.analysis.Near(naselja_bez_objekta, objekti_trenutnog_tipa)
    print(f"  - Izračunata je udaljenost (NEAR_DIST i NEAR_FID).")

    # DODAVANJE IMENA NAJBLIŽE LOKACIJE
    # 4. "Obogaćujemo" tačke objekata nazivima naselja
    objekti_sa_lokacijom = f"{objekti_trenutnog_tipa}_lokacija"
    arcpy.analysis.SpatialJoin(objekti_trenutnog_tipa, "Naselja_Sombora_Layer", objekti_sa_lokacijom)

    # 5. Kreiramo Python rečnik (dictionary)
    fid_to_name_map = {}
    with arcpy.da.SearchCursor(objekti_sa_lokacijom, ["TARGET_FID", naziv_kolone_u_naseljima]) as cursor:
        for row in cursor:
            fid_to_name_map[row[0]] = row[1]

    # 6. Dodajemo novo polje za naziv lokacije i popunjavamo ga
    novo_polje_lokacija = "NajblLok" 
    arcpy.management.AddField(naselja_bez_objekta, novo_polje_lokacija, "TEXT", field_length=50)
    
    with arcpy.da.UpdateCursor(naselja_bez_objekta, ["NEAR_FID", novo_polje_lokacija]) as cursor:
        for row in cursor:
            nearest_fid = row[0]
            location_name = fid_to_name_map.get(nearest_fid, "N/A")
            row[1] = location_name
            cursor.updateRow(row)
            
    print(f"  - Dodata je kolona '{novo_polje_lokacija}' sa nazivom najbliže lokacije.")
    
    # Brisanje privremenog lejera
    arcpy.management.Delete(objekti_sa_lokacijom)

print("\n--- Analiza je završena za sve tipove objekata! ---")

Pronađeni tipovi objekata za analizu: ['Policija', 'Bolnica', 'Škola', 'Banka', 'Pošta', 'Apoteka']

--- Početak analize dostupnosti za sve tipove objekata ---

>>> Započinjem obradu za tip: 'Policija'
  - Izdvojeni su objekti tipa 'Policija'.
  - Pronađena su naselja bez objekta tipa 'Policija'.
  - Izračunata je udaljenost (NEAR_DIST i NEAR_FID).
  - Dodata je kolona 'NajblLok' sa nazivom najbliže lokacije.

>>> Započinjem obradu za tip: 'Bolnica'
  - Izdvojeni su objekti tipa 'Bolnica'.
  - Pronađena su naselja bez objekta tipa 'Bolnica'.
  - Izračunata je udaljenost (NEAR_DIST i NEAR_FID).
  - Dodata je kolona 'NajblLok' sa nazivom najbliže lokacije.

>>> Započinjem obradu za tip: 'Škola'
  - Izdvojeni su objekti tipa 'Škola'.
  - Pronađena su naselja bez objekta tipa 'Škola'.
  - Izračunata je udaljenost (NEAR_DIST i NEAR_FID).
  - Dodata je kolona 'NajblLok' sa nazivom najbliže lokacije.

>>> Započinjem obradu za tip: 'Banka'
  - Izdvojeni su objekti tipa 'Banka'.
  - Pronađena s

# IZRAČUNAVANJE BROJA OPJEKATA PO NASELJIMA

## 5.Dodavanje novih kolona u tabelu gde se prikazuje tačan broj objekata po naseljima

In [76]:
import arcpy

print("--- Započinjem dodavanje kolone sa nazivom najbližeg mesta ---")

# Kreiranje "mape" koja povezuje ID svakog objekta sa imenom naselja

naziv_kolone_u_naseljima = "naselje__1"

# Prvo, spajamo SVE javne objekte sa naseljima da bi svaki objekat "znao" u kom se naselju nalazi.
objekti_sa_info_naselja = "Privremeni_Svi_Objekti_Sa_Lokacijom"
arcpy.analysis.SpatialJoin("Javni_Objekti_Sombor", "Naselja_Sombora", objekti_sa_info_naselja)

# Sada pravimo Python rečnik (dictionary). On je kao telefonski imenik:
# pitamo ga za ID objekta (NEAR_FID), a on vrati ime naselja.
mapa_id_u_ime_naselja = {}
with arcpy.da.SearchCursor(objekti_sa_info_naselja, ["TARGET_FID", naziv_kolone_u_naseljima]) as cursor:
    for red in cursor:
        id_objekta = red[0]
        ime_naselja = red[1]
        mapa_id_u_ime_naselja[id_objekta] = ime_naselja

print("  - Mapa 'ID Objekta -> Naziv Naselja' je uspešno kreirana.")

# Brišemo privremeni lejer jer nam više ne treba
arcpy.management.Delete(objekti_sa_info_naselja)


# Pronalaženje svih "Naselja_Bez_..." lejera koje treba ažurirati

# Automatski pronalazimo sve rezultate analize u geobazi
arcpy.env.workspace = arcpy.mp.ArcGISProject("CURRENT").defaultGeodatabase
lista_lejer_za_azuriranje = arcpy.ListFeatureClasses("Naselja_Bez_*")

print(f"\n  - Pronađeni lejeri za ažuriranje: {lista_lejer_za_azuriranje}")


# Ažuriranje svakog pronađenog lejera

for ime_lejera in lista_lejer_za_azuriranje:
    print(f"\n>>> Ažuriram lejer: '{ime_lejera}'")
    
    # Definišemo ime nove kolone (10 karaktera max)
    ime_nove_kolone = "NajblLok"
    
    # 1. Dodajemo novu kolonu, ali samo ako već ne postoji
    if not arcpy.ListFields(ime_lejera, ime_nove_kolone):
        arcpy.management.AddField(ime_lejera, ime_nove_kolone, "TEXT", field_length=50)
        print(f"    - Dodata nova kolona: '{ime_nove_kolone}'")
    else:
        print(f"    - Kolona '{ime_nove_kolone}' već postoji, biće samo prepisana.")

    # 2. Popunjavamo novu kolonu
    # Prolazimo kroz tabelu red po red
    with arcpy.da.UpdateCursor(ime_lejera, ["NEAR_FID", ime_nove_kolone]) as cursor:
        for red in cursor:
            # Uzimamo ID najbližeg objekta iz kolone NEAR_FID
            id_najblizeg_objekta = red[0]
            
            # Pomoću tog ID-a, u našoj "mapi" pronalazimo ime naselja
            ime_naselja = mapa_id_u_ime_naselja.get(id_najblizeg_objekta, "Nije pronađeno")
            
            # Upisujemo pronađeno ime u našu novu kolonu
            red[1] = ime_naselja
            cursor.updateRow(red)
            
    print(f"    - Kolona '{ime_nove_kolone}' je uspešno popunjena.")

print("\n--- AŽURIRANJE JE ZAVRŠENO! ---")

--- Započinjem dodavanje kolone sa nazivom najbližeg mesta ---
  - Mapa 'ID Objekta -> Naziv Naselja' je uspešno kreirana.

  - Pronađeni lejeri za ažuriranje: []

--- AŽURIRANJE JE ZAVRŠENO! ---


# DODAVANJE PUTEVA

In [59]:
# Isecanje (Clip) OSM podataka za puteve 

putanja_do_osm_puteva = r"C:\Users\LENOVO\Desktop\FAKS\IV GODINA\NMPOP\Sombor_objekti\gisSrbija\gis_osm_roads_free_1.shp"

# Isecamo puteve prema granicama opštine Sombor
arcpy.analysis.Clip(putanja_do_osm_puteva, "Opstina_Sombor", "Putevi_Sombor")

print("  - Korak 1/3: Putna mreža uspešno isečena na teritoriju opštine Sombor.")


# Prevođenje tipova puteva na srpski jezik 

# Dodajemo novo polje "Tip_Puta" 
arcpy.management.AddField("Putevi_Sombor", "Tip_Puta", "TEXT", field_length=50)

# Definišemo funkciju za prevod OSM 'fclass' kodova za puteve
kod_za_prevod_puteva = """
def prevedi_tip_puta(fclass):
    mapa_prevoda = {
        'motorway': 'Autoput',
        'primary': 'Glavni put',
        'secondary': 'Regionalni put',
        'tertiary': 'Lokalni put',
        'residential': 'Ulica (naselje)',
        'living_street': 'Stambena ulica',
        'unclassified': 'Neklasifikovan put',
        'service': 'Servisni put',
        'track': 'Poljski/Šumski put',
        'path': 'Staza'
    }
    # .get() će vratiti vrednost ako je nađe, ili 'Ostalo' ako je ne nađe
    return mapa_prevoda.get(fclass, 'Ostalo')
"""

# Pokrećemo Calculate Field da popunimo novu kolonu
arcpy.management.CalculateField("Putevi_Sombor", "Tip_Puta", "prevedi_tip_puta(!fclass!)", "PYTHON3", kod_za_prevod_puteva)

print("  - Korak 2/3: Tipovi puteva prevedeni na srpski jezik u koloni 'Tip_Puta'.")


# Izračunavanje dužine puteva 

# Dodajemo novu kolonu za dužinu u kilometrima
arcpy.management.AddField("Putevi_Sombor", "Duzina_KM", "DOUBLE")

# Koristimo Calculate Geometry da automatski izračunamo dužinu
arcpy.management.CalculateGeometryAttributes("Putevi_Sombor", [["Duzina_KM", "LENGTH_GEODESIC"]], "KILOMETERS")

print("  - Korak 3/3: Izračunata je dužina za svaku deonicu puta u kilometrima.")
print("\n--- Obrada putne mreže je ZAVRŠENA! ---")

  - Korak 1/3: Putna mreža uspešno isečena na teritoriju opštine Sombor.
  - Korak 2/3: Tipovi puteva prevedeni na srpski jezik u koloni 'Tip_Puta'.
  - Korak 3/3: Izračunata je dužina za svaku deonicu puta u kilometrima.

--- Obrada putne mreže je ZAVRŠENA! ---


# SIMBOLOGIJA

In [2]:
# Pristupanje trenutnom projektu i mapi/karti
aprx = arcpy.mp.ArcGISProject("CURRENT")
karta = aprx.activeMap


In [21]:
# Podešavanje POLIGONSKOG lejera (Simple Renderer / Single Symbol)
try:
    opstina_lejer = karta.listLayers("Opstina_Sombor_Layer")[0]
    simbologija = opstina_lejer.symbology
    
    if hasattr(simbologija, 'renderer'):
        simbol = simbologija.renderer.symbol
        
        simbol.color = {'RGB': [211, 211, 211, 50]} 
        
        opstina_lejer.symbology = simbologija
        print("  - Simbologija za 'Opstina_Sombor' je uspešno podešena.")

except IndexError:
    print("  - UPOZORENJE: Lejer 'Opstina_Sombor_Layer' nije pronađen na mapi.")

  - Simbologija za 'Opstina_Sombor' je uspešno podešena.


In [9]:
# Podešavanje LINIJSKOG lejera (Unique Values Renderer) ---

try:
    putevi_lejer = karta.listLayers("Putevi_Sombor")[0]
    simbologija = putevi_lejer.symbology
    
    if simbologija.renderer.type == "UniqueValueRenderer":
        simbologija.renderer.fields = ["Tip_Puta"]
        
        # Prolazimo kroz svaku grupu (svaki tip puta) i menjamo simbol
        for grupa in simbologija.renderer.groups:
            if grupa.heading[0][0] == 'Glavni put':
                grupa.symbol.color = {'RGB': [255, 0, 0, 100]} 
                grupa.symbol.width = 2
            elif grupa.heading[0][0] == 'Regionalni put':
                grupa.symbol.color = {'RGB': [255, 165, 0, 100]} 
                grupa.symbol.width = 1.5
            elif grupa.heading[0][0] == 'Lokalni put':
                grupa.symbol.color = {'RGB': [80, 80, 80, 100]} 
                grupa.symbol.width = 1
            elif grupa.heading[0][0] == 'Ulica (naselje)':
                 grupa.symbol.color = {'RGB': [150, 150, 150, 100]} 
                 grupa.symbol.width = 0.5
          
        
        putevi_lejer.symbology = simbologija
        print("  - Simbologija za 'Putevi_Sombor' je uspešno podešena.")

except IndexError:
    print("  - UPOZORENJE: Lejer 'Putevi_Sombor' nije pronađen na mapi.")


  - Simbologija za 'Putevi_Sombor' je uspešno podešena.


In [20]:
# Podešavanje TAČKASTOG lejera (Graduated Symbols Renderer)
# Lejer: Naselja_Bez_Apoteke (kao primer)

lista_lejer_za_obradu = karta.listLayers("Naselja_Bez_*")

if not lista_lejer_za_obradu:
    print("  - Nisu pronađeni 'Naselja_Bez_*' lejeri na mapi za podešavanje.")
else:
    try:
        color_ramp = aprx.listColorRamps('Brown-Green (5 Classes)')[0]
        
        for lejer in lista_lejer_za_obradu:
            print(f"    -> Podešavam '{lejer.name}'")
            simbologija = lejer.symbology
            simbologija.updateRenderer("GraduatedColorsRenderer")
            simbologija.renderer.classificationField = "NEAR_DIST"
            simbologija.renderer.breakCount = 5
            simbologija.renderer.colorRamp = color_ramp
            lejer.symbology = simbologija
            
    except IndexError:
        print("  - !!! GREŠKA: Paleta boja 'Brown-Green (5 Classes)' nije pronađena u stilovima projekta.")

print("\n--- Podešavanje simbologije je završeno! ---")

    -> Podešavam 'Naselja_Bez_Apoteka'
    -> Podešavam 'Naselja_Bez_Banka'
    -> Podešavam 'Naselja_Bez_Bolnica'
    -> Podešavam 'Naselja_Bez_Policija'
    -> Podešavam 'Naselja_Bez_Po_ta'
    -> Podešavam 'Naselja_Bez__kola'

--- Podešavanje simbologije je završeno! ---


In [24]:
# 1. Objekti_Apoteka (Crvena boja)
try:
    lejer = karta.listLayers("Objekti_Apoteka")[0]
    simbologija = lejer.symbology
    simbol = simbologija.renderer.symbol
    simbol.applySymbolFromGallery('Pharmacy')
    simbol.color = {'RGB': [255, 0, 0, 100]} # Crvena
    
    lejer.symbology = simbologija
    print("  - Simbologija za 'Objekti_Apoteka' je podešena.")
except Exception as e:
    print(f"  - GREŠKA pri obradi 'Objekti_Apoteka': {e}")

  - Simbologija za 'Objekti_Apoteka' je podešena.


In [25]:
# 2. Objekti_Banka (Zelena boja) 
try:
    lejer = karta.listLayers("Objekti_Banka")[0]
    simbologija = lejer.symbology
    simbol = simbologija.renderer.symbol
    simbol.applySymbolFromGallery('Bank')
    simbol.color = {'RGB': [0, 128, 0, 100]} # Tamno zelena
    lejer.symbology = simbologija
    print("  - Simbologija za 'Objekti_Banka' je podešena.")
except Exception as e:
    print(f"  - GREŠKA pri obradi 'Objekti_Banka': {e}")

  - Simbologija za 'Objekti_Banka' je podešena.


In [32]:
# 3. Objekti_Bolnica (Plava boja) 
try:
    lejer = karta.listLayers("Objekti_Bolnica")[0]
    simbologija = lejer.symbology
    simbol = simbologija.renderer.symbol
    simbol.applySymbolFromGallery('Hospital')
    simbol.color = {'RGB': [0, 0, 255, 100]} # Plava
    lejer.symbology = simbologija
    print("  - Simbologija za 'Objekti_Bolnica' je podešena.")
except Exception as e:
    print(f"  - GREŠKA pri obradi 'Objekti_Bolnica': {e}")

  - Simbologija za 'Objekti_Bolnica' je podešena.


In [28]:
# 4. Objekti_Policija (Svetlo plava boja)
try:
    lejer = karta.listLayers("Objekti_Policija")[0]
    simbologija = lejer.symbology
    simbol = simbologija.renderer.symbol
    simbol.applySymbolFromGallery('Police Station')
    simbol.color = {'RGB': [0, 112, 255, 100]} # Svetlo plava
    lejer.symbology = simbologija
    print("  - Simbologija za 'Objekti_Policija' je podešena.")
except Exception as e:
    print(f"  - GREŠKA pri obradi 'Objekti_Policija': {e}")

  - Simbologija za 'Objekti_Policija' je podešena.


In [29]:
# 5. Objekti_Po_ta (Narandžasta boja) 
try:
    lejer = karta.listLayers("Objekti_Po_ta")[0]
    simbologija = lejer.symbology
    simbol = simbologija.renderer.symbol
    simbol.applySymbolFromGallery('Post Office')
    simbol.color = {'RGB': [255, 165, 0, 100]} # Narandžasta
    lejer.symbology = simbologija
    print("  - Simbologija za 'Objekti_Po_ta' je podešena.")
except Exception as e:
    print(f"  - GREŠKA pri obradi 'Objekti_Po_ta': {e}")

  - Simbologija za 'Objekti_Po_ta' je podešena.


In [33]:
# 6. Objekti_kola (Ljubičasta boja)
try:
    lejer = karta.listLayers("Objekti__kola")[0]
    simbologija = lejer.symbology
    simbol = simbologija.renderer.symbol
    simbol.applySymbolFromGallery('School')
    simbol.color = {'RGB': [128, 0, 128, 100]} # Ljubičasta
    lejer.symbology = simbologija
    print("  - Simbologija za 'Objekti_kola' je podešena.")
except Exception as e:
    print(f"  - GREŠKA pri obradi 'Objekti_kola': {e}")


print("\n--- Podešavanje simbologije tačkastih objekata je završeno! ---")

  - Simbologija za 'Objekti_kola' je podešena.

--- Podešavanje simbologije tačkastih objekata je završeno! ---
